In [1]:
!pip install python-telegram-bot --quiet
!pip install opencv-python --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.3/702.3 kB 9.1 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
TELEGRAM_TOKEN = 'YOUR_BOT_TOKEN_HERE'  # Replace this!


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from telegram import Update
from telegram.ext import ApplicationBuilder, MessageHandler, filters, ContextTypes

# Load model
model = tf.keras.models.load_model("brain_tumor_model.h5")  # or .keras
IMG_SIZE = 128
labels = ["glioma", "meningioma", "pituitary", "notumor"]

# Preprocess image
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    img = img.reshape(1, IMG_SIZE, IMG_SIZE, 1)
    return img

# Handle image messages
async def handle_image(update: Update, context: ContextTypes.DEFAULT_TYPE):
    photo = await update.message.photo[-1].get_file()
    photo_path = await photo.download_to_drive()
    img = preprocess_image(photo_path)
    prediction = model.predict(img)[0]
    predicted_label = labels[np.argmax(prediction)]
    confidence = round(np.max(prediction) * 100, 2)

    await update.message.reply_text(
        f"🧠 Prediction: *{predicted_label.capitalize()}*\n🔍 Confidence: *{confidence}%*",
        parse_mode="Markdown"
    )

# Build bot
app = ApplicationBuilder().token(TELEGRAM_TOKEN).build()
app.add_handler(MessageHandler(filters.PHOTO, handle_image))

# Start bot (in Colab-safe way)
import nest_asyncio
nest_asyncio.apply()
await app.run_polling()
